In [54]:
from torchvision import datasets
import torchvision
import torchvision.transforms as transforms
import numpy as np
import torch 
import torch.nn as nn
import torch.nn.functional as F

In [55]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5,0.5,0.5))])

In [56]:
train_data = datasets.CIFAR10(root = 'data', train = True, download = True, transform = transform)
test_data = datasets.CIFAR10(root = 'data', train = False, download = True, transform = transform)

Files already downloaded and verified
Files already downloaded and verified


In [57]:
dev_size = 0.2
idx = list(range(len(train_data)))
np.random.shuffle(idx)
split_size = int(len(idx) * dev_size)
train_idx = idx[split_size:]
dev_idx = idx[:split_size]

In [58]:
from torch.utils.data import SubsetRandomSampler, DataLoader

In [59]:
batch_size = 100
train_sampler = SubsetRandomSampler(train_idx)
dev_sampler = SubsetRandomSampler(dev_idx)

In [60]:
train_loader = DataLoader(train_data, batch_size = batch_size, sampler = train_sampler)
dev_loader = DataLoader(train_data, batch_size = batch_size, sampler = dev_sampler)
test_loader = DataLoader(test_data, batch_size = batch_size)

In [61]:
class CNN_model(nn.Module):
    def __init__(self):
        super(CNN_model,self).__init__()
        self.conv1 = nn.Conv2d(3, 10, 3, 1, 1)
        self.conv2 = nn.Conv2d(10, 20, 3, 1,1)
        self.conv3 = nn.Conv2d(20, 40, 3, 1, 1)
        self.pool = nn.MaxPool2d(2,2)
        self.dropout = nn.Dropout(0.2)
        self.linear1 = nn.Linear(4 * 4 * 40,100)
        self.linear2 = nn.Linear(100,10)
    def forward(self,x):
        z = F.relu(self.conv1(x))
        z = self.pool(z)
        z = F.relu(self.conv2(z))
        z = self.pool(z)
        z = F.relu(self.conv3(z))
        z = self.pool(z)
        z = z.view(-1, 4 * 4 * 40)
        z = self.dropout(z)
        z = F.relu(self.linear1(z))
        z = self.dropout(z)
        z = F.log_softmax(self.linear2(z), dim = 1)
        return  z

In [62]:
model = CNN_model()

loss_function = nn.NLLLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 50

In [63]:
train_loss_arr, train_acc_arr, dev_loss_arr, dev_acc_arr = [],[],[],[]
for e in range(epochs):
    running_acc, running_loss = 0,0
    iterations = 0
    model.train()
    for data, target in train_loader:
        iterations += 1
        pred = model(data)
      
        loss = criterion(pred,target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        ps = torch.exp(pred)
        top_p, top_class = ps.topk(1, dim = 1)
        running_acc += accuracy_score(target, top_class)
        running_loss += loss.item()
    with torch.no_grad():
        cnt = 0
        dev_acc = 0
        dev_loss = 0;
        for X_batch, y_batch in dev_loader:
            cnt+=1
            pred = model(X_batch)
            dev_loss += criterion(pred, y_batch).item()
            ps_dev = torch.exp(pred)
            top_p, top_class = ps_dev.topk(1, dim = 1)
            dev_acc += accuracy_score(y_batch, top_class)
    train_loss_arr.append(running_loss/iterations)
    train_acc_arr.append(running_acc/iterations)
    dev_loss_arr.append(dev_loss/cnt)
    dev_acc_arr.append(dev_acc/cnt)
    print("Epoch {}/{}..".format(e, epochs),
          "Training Loss: {:.3f}..".format(running_loss/iterations),
          "Validation Loss: {:.3f}..".format(dev_loss/cnt),
          "Training Acc: {:.3f}..".format(running_acc/iterations),
          "Validation Acc: {:.3f}..".format(dev_acc/cnt))
         
        
    

KeyboardInterrupt: 

In [64]:
torch.cuda.is_available()

False